In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

In [24]:
supplier = ['AUS', 'CHN', 'KAZ']
plant = ['CHT', 'ATX']
DC = ['HTX', 'NYC', 'LA']
mkt = ['APAC', 'EU', 'NA', 'SA', 'ME']

demand_mkt = {
    'APAC': 15000,
    'EU': 10000,
    'NA': 8000,
    'ME': 4000,
    'SA': 3000
}

data = {
    '': ['ATX', 'CHT', 'APAC', 'EU', 'NA', 'ME', 'SA'],
    'HTX': [20, 90, 220, 195, 70, 240, 90],
    'NYC': [100, 30, 250, 180, 75, 230, 115],
    'LA': [70, 120, 200, 210, 80, 235, 95]
}

# Create the DataFrame
dc_mkt_cost = pd.DataFrame(data)
dc_mkt_cost.set_index('', inplace=True)
dc_mkt_cost.index.name = None  # Remove the name of the index


data_2 = {
    '': ['ATX', 'CHT'],
    'CHN': [150, 160],
    'AUS': [175, 170],
    'KAZ': [165, 180]
    }
supplier_plant_cost = pd.DataFrame(data_2)
supplier_plant_cost.set_index('', inplace=True)
supplier_plant_cost.index.name = None

In [25]:
construction_cost = {
    'HTX': 1500000,
    'NYC': 2000000,
    'LA': 1800000
}

In [26]:
m = gp.Model()

y = m.addVars(DC, vtype=GRB.BINARY, name="DC Location")

a = m.addVars(supplier, plant, lb=0, name="Supplier to Plant")
b = m.addVars(plant, DC, lb=0, name="Plant to DC")
c = m.addVars(DC, mkt, lb=0, name="Plant to MKT")
m.update()

In [27]:
plant_cap = m.addConstrs((gp.quicksum(a[i, j] for i in supplier) <= 30000 for j in plant), name="Plant Capacity")
dc_cap = m.addConstrs((gp.quicksum(b[j, k] for j in plant) <= 40000 for k in DC), name="DC Capacity")
demand_cons = m.addConstrs((gp.quicksum(c[k, l] for k in DC) >= demand_mkt[l] for l in mkt), name="Demand in MKT")

plant_transshipment = m.addConstrs((gp.quicksum(a[i, j] for i in supplier) >= gp.quicksum(b[j, k] for k in DC) for j in plant), name="Plant Transshipment")
dc_transshipment = m.addConstrs((gp.quicksum(b[j, k] for j in plant) >= gp.quicksum(c[k, l] for l in mkt) for k in DC), name="DC Transshipment")

m.update()

In [23]:
dc_mkt_cost.loc['']

,HTX,NYC,LA
ATX,20,100,70
CHT,90,30,120
APAC,220,250,200
EU,195,180,210
NA,70,75,80
ME,240,230,235
SA,90,115,95


In [28]:
m.setObjective(gp.quicksum(supplier_plant_cost.loc[j, i] * a[i, j] for i in supplier for j in plant) + gp.quicksum(dc_mkt_cost.loc[l, k] * c[k, l] for k in DC for l in mkt) + gp.quicksum(construction_cost[k] * y[k] for k in DC), GRB.MINIMIZE)
m.update()

In [29]:
m.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 7 5800HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 15 rows, 30 columns and 60 nonzeros
Model fingerprint: 0x9b35756a
Variable types: 27 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+01, 2e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+03, 4e+04]
Presolve removed 5 rows and 9 columns
Presolve time: 0.01s
Presolved: 10 rows, 21 columns, 42 nonzeros
Variable types: 21 continuous, 0 integer (0 binary)

Root relaxation: objective 1.265000e+07, 14 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    1.265000e+07 1.2650e+07  0.

In [30]:
for v in m.getVars():
    if v.X > 0:
        print(v.VarName, v.X)

Supplier to Plant[CHN,CHT] 10000.0
Supplier to Plant[CHN,ATX] 30000.0
Plant to DC[CHT,HTX] 10000.0
Plant to DC[ATX,HTX] 1000.0
Plant to DC[ATX,NYC] 14000.0
Plant to DC[ATX,LA] 15000.0
Plant to MKT[HTX,NA] 8000.0
Plant to MKT[HTX,SA] 3000.0
Plant to MKT[NYC,EU] 10000.0
Plant to MKT[NYC,ME] 4000.0
Plant to MKT[LA,APAC] 15000.0
